In [1]:
# import dependencies
from bs4 import BeautifulSoup as soup
from splinter import Browser
import pandas as pd
import requests
import numpy as np
import sqlite3
import os
import datetime
import time

# Scraper Code

### Single Scrape (dev code)

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# define the url and visit it with browser
storage_url = "http://www.gasnom.com/ip/freebird/oauc.cfm?type=1"
browser.visit(storage_url)

# grab the table data
tables = pd.read_html(storage_url)
storage_info = tables[0]
storage_raw = tables[2]

# reformat the table data
tsp_name = storage_info.iloc[0][1]
posting_date = storage_info.iloc[2][1]

storage_raw.columns = ["location_name","loc","loc_purp_desc","loc_qti","flow_ind","all_qty_avail",\
                       "design_capacity", "operating_capacity","tsq","oac","it_indicator","qty_reason","blanks"]
storage_raw = storage_raw.fillna(0)
storage_cl1 = storage_raw.loc[storage_raw['location_name'] != 0]
storage_cl1 = storage_cl1.drop(storage_cl1.index[0])
storage_cl1 = storage_cl1.reset_index(drop=True)
storage_cl2 = storage_cl1.iloc[:,[1,8]]

storage_cl2.insert(0, 'date', posting_date)
storage_cl2.insert(1, 'tsp_name', tsp_name)
storage_cl2

,date,tsp_name,loc,tsq
0,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,0002,0
1,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,021039,4100
2,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,012709,-649


### Scrape Loop

#### Scrape Today's Data

In [3]:
storage_names = ['caledonia', 'freebird', 'baygas','eastcheyenne']
column_names =['date','tsp_name',"loc","tsq"]

storage_df = pd.DataFrame(columns=column_names)

base_url = "http://www.gasnom.com/ip/"
end_url = "/oauc.cfm?type=1"

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

for name in storage_names :
    # define the url and visit it with browser
    storage_url = base_url + name + end_url
    browser.visit(storage_url)
    
    # grab the table data
    tables = pd.read_html(storage_url)
    storage_info = tables[0]
    storage_raw = tables[2]

    # reformat the table data
    tsp_name = storage_info.iloc[0][1]
    posting_date = storage_info.iloc[2][1]

    storage_raw.columns = ["location_name","loc","loc_purp_desc","loc_qti","flow_ind","all_qty_avail",\
                           "design_capacity", "operating_capacity","tsq","oac","it_indicator","qty_reason","blanks"]
    storage_raw = storage_raw.fillna(0)
    storage_cl1 = storage_raw.loc[storage_raw['location_name'] != 0]
    storage_cl1 = storage_cl1.drop(storage_cl1.index[0])
    storage_cl1 = storage_cl1.reset_index(drop=True)
    storage_cl2 = storage_cl1.iloc[:,[1,8]]

    storage_cl2.insert(0, 'date', posting_date)
    storage_cl2.insert(1, 'tsp_name', tsp_name)
    storage_df = storage_df.append(storage_cl2)

browser.quit()
storage_df

,date,tsp_name,loc,tsq
0,"October 10, 2020 09:05 PM CT","Caledonia Energy Partners, L.L.C.",542519,-2420
1,"October 10, 2020 09:05 PM CT","Caledonia Energy Partners, L.L.C.",542524,4950
0,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,0002,0
1,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,021039,4100
2,"October 10, 2020 09:05 PM CT",Freebird Gas Storage L.L.C.,012709,-649
0,"October 10, 2020 09:05 PM CT",BayGas Storage,BG-1002,-120961
1,"October 10, 2020 09:05 PM CT",BayGas Storage,BG-1003,7450
2,"October 10, 2020 09:05 PM CT",BayGas Storage,BG-1008,0
3,"October 10, 2020 09:05 PM CT",BayGas Storage,BG-1013,149670
0,"October 10, 2020 09:05 PM CT",East Cheyenne Gas Storage,029734466,7424


#### Scrape Historic Data

In [3]:
# create the empty df
column_names =['date','tsp_name',"loc","tsq"]
storage_df = pd.DataFrame(columns=column_names)

# create the list of storages to loop through
storage_names = ['caledonia', 'freebird', 'baygas','eastcheyenne']

# set the start and end date for the loop
start = datetime.datetime(2017,9,30)
end = datetime.datetime(2020,10,13)
datecount = end - start
dates = [(start - datetime.timedelta(days=x)).strftime('%m/%d/%Y') for x in range(-datecount, 0)]

# url config
base_url = "http://www.gasnom.com/ip/"
mid_url = "/oauc.cfm?dt="
end_url = "&type=1"

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

for name in storage_names :
    for date in dates :
    # define the url and visit it with browser
        storage_url = base_url + name + mid_url + date + end_url
        print(storage_url)
        browser.visit(storage_url)

        # grab the table data
        tables = pd.read_html(storage_url)
        storage_info = tables[0]
        storage_raw = tables[2]

        # reformat the table data
        tsp_name = storage_info.iloc[0][1]
        posting_date = storage_info.iloc[2][1]

        storage_raw.columns = ["location_name","loc","loc_purp_desc","loc_qti","flow_ind","all_qty_avail",\
                               "design_capacity", "operating_capacity","tsq","oac","it_indicator","qty_reason","blanks"]
        storage_raw = storage_raw.fillna(0)
        storage_cl1 = storage_raw.loc[storage_raw['location_name'] != 0]
        storage_cl1 = storage_cl1.drop(storage_cl1.index[0])
        storage_cl1 = storage_cl1.reset_index(drop=True)
        storage_cl2 = storage_cl1.iloc[:,[1,8]]

        storage_cl2.insert(0, 'date', posting_date)
        storage_cl2.insert(1, 'tsp_name', tsp_name)
        storage_df = storage_df.append(storage_cl2)
        
        time.sleep(3) # Sleep for 3 seconds

browser.quit()
storage_df
        

http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=10/08/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=10/07/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=10/06/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=10/05/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=10/04/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=10/03/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=10/02/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=10/01/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/30/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/29/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/28/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/27/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/26/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/25/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/24/2020&type=1
http://www.gasnom.com/ip/

http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=06/03/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=06/02/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=06/01/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/31/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/30/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/29/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/28/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/27/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/26/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/25/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/24/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/23/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/22/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/21/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/20/2020&type=1
http://www.gasnom.com/ip/

http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/28/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/27/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/26/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/25/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/24/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/23/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/22/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/21/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/20/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/19/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/18/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/17/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/16/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/15/2020&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=01/14/2020&type=1
http://www.gasnom.com/ip/

http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/23/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/22/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/21/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/20/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/19/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/18/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/17/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/16/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/15/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/14/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/13/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/12/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/11/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/10/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=09/09/2019&type=1
http://www.gasnom.com/ip/

http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/19/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/18/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/17/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/16/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/15/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/14/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/13/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/12/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/11/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/10/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/09/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/08/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/07/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/06/2019&type=1
http://www.gasnom.com/ip/caledonia/oauc.cfm?dt=05/05/2019&type=1
http://www.gasnom.com/ip/

...

http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/19/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/18/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/17/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/16/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/15/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/14/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/13/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/12/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/11/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/10/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/09/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/08/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/07/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/06/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?d

http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/20/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/19/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/18/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/17/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/16/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/15/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/14/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/13/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/12/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/11/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/10/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/09/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/08/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=08/07/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?d

http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/21/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/20/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/19/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/18/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/17/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/16/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/15/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/14/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/13/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/12/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/11/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/10/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/09/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=04/08/2018&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?d

http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/21/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/20/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/19/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/18/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/17/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/16/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/15/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/14/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/13/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/12/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/11/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/10/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/09/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?dt=12/08/2017&type=1
http://www.gasnom.com/ip/eastcheyenne/oauc.cfm?d

,date,tsp_name,loc,tsq
0,"October 8, 2020 10:05 PM CT","Caledonia Energy Partners, L.L.C.",542519,-34420
1,"October 8, 2020 10:05 PM CT","Caledonia Energy Partners, L.L.C.",542524,0
0,"October 7, 2020 10:05 PM CT","Caledonia Energy Partners, L.L.C.",542519,-35020
1,"October 7, 2020 10:05 PM CT","Caledonia Energy Partners, L.L.C.",542524,0
0,"October 6, 2020 10:05 PM CT","Caledonia Energy Partners, L.L.C.",542519,-12420
...,...,...,...,...
0,"October 5, 2017 10:05 PM CT",East Cheyenne Gas Storage,029734466,38963
0,"October 4, 2017 10:05 PM CT",East Cheyenne Gas Storage,029734466,72207
0,"October 3, 2017 10:05 PM CT",East Cheyenne Gas Storage,029734466,66827
0,"October 2, 2017 10:05 PM CT",East Cheyenne Gas Storage,029734466,74999


# Database Code

In [19]:
clean_df = storage_df.drop_duplicates(subset=['date', 'loc'])

In [21]:
conn = sqlite3.connect('scraped_storage_data.sqlite')
c = conn.cursor()

In [23]:
clean_df.to_sql('storageData', conn, if_exists='append', index = False)

In [24]:
c.execute("SELECT * FROM storageData")

rows = c.fetchall()

for row in rows:
    print(row)

('October 8, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-34420')
('October 8, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '0')
('October 7, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-35020')
('October 7, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '0')
('October 6, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-12420')
('October 6, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '0')
('October 5, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-7420')
('October 5, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '0')
('October 4, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-2420')
('October 4, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '5102')
('October 3, 2020 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-2420')
('October 3, 2020 10:05 PM CT', 'Caledonia Ene

('March 4, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-157467')
('March 4, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '2419')
('March 3, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-152467')
('March 3, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '2420')
('March 2, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-150467')
('March 2, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '2420')
('March 1, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-51872')
('March 1, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '2743')
('February 28, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-111313')
('February 28, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542524', '934')
('February 27, 2019 10:05 PM CT', 'Caledonia Energy Partners, L.L.C.', '542519', '-108571')
('February 27, 2019 10:05 PM CT', 'Ca

('September 11, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '0002', '0')
('September 11, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '021039', '25917')
('September 11, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '012709', '-36334')
('September 10, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '0002', '0')
('September 10, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '021039', '25917')
('September 10, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '012709', '-69334')
('September 9, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '0002', '0')
('September 9, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '021039', '51787')
('September 9, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '012709', '0')
('September 8, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '0002', '0')
('September 8, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '021039', '51787')
('September 8, 2019 10:05 PM CT', 'Freebird Gas Storage L.L.C.', '012709', '0')
('September 7, 2

('August 2, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1008', '0')
('August 2, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1013', '234380')
('August 1, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1002', '-73125')
('August 1, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1003', '21937')
('August 1, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1008', '0')
('August 1, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1013', '234380')
('July 31, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1002', '-116336')
('July 31, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1003', '39301')
('July 31, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1008', '0')
('July 31, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1013', '211384')
('July 30, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1002', '-102900')
('July 30, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1003', '43501')
('July 30, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1008', '0')
('July 30, 2020 10:05 PM CT', 'BayGas Storage', 'BG-1013', '206384')
('July 29, 2020 10:05 PM CT', 'BayGas Storage', '

('July 10, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1002', '-170066')
('July 10, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1003', '27600')
('July 10, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1008', '0')
('July 10, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1013', '276742')
('July 9, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1002', '-188468')
('July 9, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1003', '29062')
('July 9, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1008', '0')
('July 9, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1013', '325269')
('July 8, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1002', '288576')
('July 8, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1003', '25799')
('July 8, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1008', '0')
('July 8, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1013', '375159')
('July 7, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1002', '246467')
('July 7, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1003', '25799')
('July 7, 2019 10:05 PM CT', 'BayGas Storage', 'BG-1008', '0'

('February 23, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-72735')
('February 22, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-72735')
('February 21, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-66908')
('February 20, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-56458')
('February 19, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-41687')
('February 18, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-88879')
('February 17, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-88879')
('February 16, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-88879')
('February 15, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-88879')
('February 14, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-61978')
('February 13, 2020 10:05 PM CT', 'East Cheyenne Gas Storage', '029734466', '-98991')
('February 12, 2020 10:05 PM CT', 'East Cheyenne Gas S

In [29]:
clean_df.to_csv('historic_storage.csv', index=False)